In [7]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from pmdarima import auto_arima


Load TSLA Data

In [8]:
# Load TSLA Data
data = yf.download('TSLA', start='2015-01-01', end='2024-10-31')


[*********************100%***********************]  1 of 1 completed


In [9]:
tsla_data = data[['Close']]
tsla_data.dropna(inplace=True)

C:\Users\Firew Ayele\AppData\Local\Temp\ipykernel_12376\2920378769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsla_data.dropna(inplace=True)


Train-Test Split

In [10]:
# Train-Test Split (80% train, 20% test)
train_size = int(len(tsla_data) * 0.8)
train, test = tsla_data[:train_size], tsla_data[train_size:]

calculate evaluation metrics

In [11]:
# Helper function to calculate evaluation metrics
def evaluate_model(test, predictions):
    mae = mean_absolute_error(test, predictions)
    rmse = np.sqrt(mean_squared_error(test, predictions))
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    return mae, rmse, mape

ARIMA Model

In [12]:
# Use auto_arima to find optimal (p, d, q) parameters
auto_arima_model = auto_arima(train, seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)
arima_order = auto_arima_model.order
print("Optimal ARIMA order:", arima_order)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=12345.046, Time=2.08 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=12340.889, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=12339.852, Time=0.19 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=12339.956, Time=0.27 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=12339.337, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=12341.053, Time=0.82 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 3.506 seconds
Optimal ARIMA order: (0, 1, 0)


Train ARIMA model

In [13]:
# Train ARIMA model
arima_model = ARIMA(train, order=arima_order)
arima_model_fit = arima_model.fit()

c:\Users\Firew Ayele\Desktop\kifiya\GMF_Investments\.venv_gmf\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Firew Ayele\Desktop\kifiya\GMF_Investments\.venv_gmf\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Firew Ayele\Desktop\kifiya\GMF_Investments\.venv_gmf\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
